In [2]:
import numpy as np
import cv2
import time

# Constants
FRAME_HEIGHT = 200
FRAME_WIDTH = 300
CALIBRATION_TIME = 30
BG_WEIGHT = 0.5
OBJ_THRESHOLD = 18

# Initialize global variables
background = None
hand = None
frames_elapsed = 0

# Define the region of interest
region_top = 0
region_bottom = int(2 * FRAME_HEIGHT / 3)
region_left = int(FRAME_WIDTH / 2)
region_right = FRAME_WIDTH

class HandData:
    def __init__(self):
        self.top = None
        self.bottom = None
        self.left = None
        self.right = None
        self.centerX = 0
        self.prevCenterX = 0
        self.isInFrame = False
        self.isWaving = False
        self.fingers = None
        self.gestureList = []

    def update(self, top, bottom, left, right):
        self.top, self.bottom, self.left, self.right = top, bottom, left, right
        # Calculate centerX here
        self.centerX = int((self.left[0] + self.right[0]) / 2)

    def check_for_waving(self, centerX):
        self.prevCenterX = self.centerX
        self.centerX = centerX
        if abs(self.centerX - self.prevCenterX) > 3:
            self.isWaving = True
        else:
            self.isWaving = False

def write_on_image(frame, text):
    global frames_elapsed  # Access global variable

    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)

    # Highlight the region of interest
    cv2.rectangle(frame, (region_left, region_top), (region_right, region_bottom), (255, 255, 255), 2)

def get_region(frame):
    # Separate the region of interest from the rest of the frame.
    region = frame[region_top:region_bottom, region_left:region_right]
    # Make it grayscale so we can detect the edges more easily.
    region = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    # Use a Gaussian blur to prevent frame noise from being labeled as an edge.
    region = cv2.GaussianBlur(region, (5, 5), 0)
    return region

def get_average(region):
    global background
    if background is None:
        background = region.copy().astype("float")
        return
    cv2.accumulateWeighted(region, background, BG_WEIGHT)

def segment(region):
    global hand
    diff = cv2.absdiff(background.astype(np.uint8), region)
    thresholded_region = cv2.threshold(diff, OBJ_THRESHOLD, 255, cv2.THRESH_BINARY)[1]

    contours, _ = cv2.findContours(thresholded_region.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        if hand is not None:
            hand.isInFrame = False
        return None, None
    else:
        if hand is not None:
            hand.isInFrame = True
        segmented_region = max(contours, key=cv2.contourArea)
        return thresholded_region, segmented_region

def get_convex_hull(segmented_image):
    convex_hull = cv2.convexHull(segmented_image)
    top = tuple(convex_hull[convex_hull[:, :, 1].argmin()][0])
    bottom = tuple(convex_hull[convex_hull[:, :, 1].argmax()][0])
    left = tuple(convex_hull[convex_hull[:, :, 0].argmin()][0])
    right = tuple(convex_hull[convex_hull[:, :, 0].argmax()][0])
    return top, bottom, left, right

def count_fingers(thresholded_image):
    # Find the height at which we will draw the line to count fingers.
    line_height = int(hand.top[1] + (0.2 * (hand.bottom[1] - hand.top[1])))
    
    # Create a mask for the line region of interest where the fingers are expected.
    line_mask = np.zeros(thresholded_image.shape[:2], dtype=np.uint8)
    
    # Draw a horizontal line across the region of interest.
    cv2.line(line_mask, (0, line_height), (thresholded_image.shape[1], line_height), 255, 1)
    
    # Use a bitwise AND to isolate the area where the fingers might be.
    line = cv2.bitwise_and(thresholded_image, thresholded_image, mask=line_mask)

    # Find contours in the line image.
    contours, _ = cv2.findContours(line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    fingers = 0
    
    # Count the fingers based on the width of the detected contours.
    for curr in contours:
        width = len(curr)
        if width < 3 * abs(hand.right[0] - hand.left[0]) / 4 and width > 5:
            fingers += 1
    
    return fingers


def most_frequent(input_list):
    dict_count = {}
    count = 0
    most_freq = 0

    for item in reversed(input_list):
        dict_count[item] = dict_count.get(item, 0) + 1
        if dict_count[item] >= count:
            count, most_freq = dict_count[item], item

    return most_freq

def main():
    global frames_elapsed, background, hand
    background = None
    hand = HandData()
    frames_elapsed = 0

    capture = cv2.VideoCapture(0)  # Change to 1 if necessary
    time.sleep(2)

    while True:
        ret, frame = capture.read()
        if not ret:
            print("Failed to capture frame")
            break

        frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
        frame = cv2.flip(frame, 1)
        region = get_region(frame)

        if frames_elapsed < CALIBRATION_TIME:
            get_average(region)
            text = "Calibrating..."
        else:
            thresholded_region, segmented_region = segment(region)
            if segmented_region is not None:
                hand.update(*get_convex_hull(segmented_region))
                hand.check_for_waving(hand.centerX)

                hand.gestureList.append(count_fingers(thresholded_region))
                if frames_elapsed % 12 == 0:
                    hand.fingers = most_frequent(hand.gestureList)
                    hand.gestureList.clear()

                text = "Waving" if hand.isWaving else "Rock" if hand.fingers == 0 else "Pointing" if hand.fingers == 1 else "Scissors" if hand.fingers == 2 else "Waving"
            else:
                hand.isInFrame = False
                text = "No hand detected"

        write_on_image(frame, text)
        cv2.imshow("Camera Input", frame)
        frames_elapsed += 1

        if cv2.waitKey(1) & 0xFF == ord('x'):
            break

    capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
